In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download("stopwords")

In [8]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import csv
from itertools import groupby
import re


mystem = Mystem()
russian_stopwords = stopwords.words("russian")

In [13]:
def load_and_prepare(file):
    comments_file = open(file)
    comments = csv.reader(comments_file, delimiter=';')
    tpl = []

    for i in comments:
        tpl.append(i)

    flat_list = []
    for sublist in tpl:
        for item in sublist:
            if item != ' ' and item != '':
                flat_list.append(item)

    my_punctuation = [punctuation]
    my_punctuation = my_punctuation[0]
    my_punctuation += '«»—−–…'

    ready_tokens = []

    for comment in flat_list:

        comment = re.sub(r'http\S+', '', comment)
        comment = re.sub(r'\[(id|club)\d*\|.*\]', '', comment)

        token_cleared = []
        for k, g in groupby(comment):
            if k not in set(my_punctuation):
                token_cleared.extend(g)
        token = ''.join(token_cleared)

        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"
                                   u"\U0001F300-\U0001F5FF"
                                   u"\U0001F680-\U0001F6FF"
                                   u"\U0001F1E0-\U0001F1FF"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001F900-\U0001F9FF"
                                   "]+", flags=re.UNICODE)
        token = emoji_pattern.sub(r'', token)

        tokens = mystem.lemmatize(token)
        tokens = [token for token in tokens if token not in russian_stopwords \
                  and token.strip() != " " and token.strip() != "" and token.strip() != "\n" \
                  and not token.isdigit()]

        if tokens:
            ready_tokens.append(tokens)

    comments_file.close()

    return ready_tokens

In [ ]:
print(load_and_prepare('group_texts.csv'))